# AMA Predictor - XGBoost Modeling
* Last Updated: 
    * <b><font color='coral'>April 15, 2024 (GH)</font></b>
    * Original Notebook (dev1 branch): 5.1_XGBoost-Final.ipynb
    * Notebook to be submitted for execution for hyperparameter tuning!
        * https://notebooks.cloud.google.com/view/prj-cxbi-dev-nane1-dsc-expe-northamerica-northeast1-notebooks/executor_files/final_xgboost_gh__1713269249954/final_xgboost_gh_at_2024_04_16_05_07_36.ipynb?_ga=2.206854810.-883769773.1667388537

In [2]:
import numpy as np
import pandas as pd

In [3]:
# df = pd.read_csv('Data_Prep_After_FE.csv')
df = pd.read_csv('gs://tf-mnb-bucket-for_gang_huang/AMA_Predictor/Data_Prep_After_FE.csv')

df.set_index('startdatetime', inplace=True)

In [4]:
df.head()

,ama,hour,minute,duration,dayofweek,dayofyear,weeknum,month,quarter,year,...,distributor_Sticky Fingers Production Inc.,distributor_TBS,distributor_Warner Brothers,distributor_eOne,cat_genre_Cat_Comedy,cat_genre_Cat_Comedy_Romance,cat_genre_Cat_Doc_Crime_Mystery,cat_genre_Cat_Other_Genre,cat_genre_Cat_Reality_Competition,cat_genre_Cat_Talk_Show
startdatetime,,,,,,,,,,,,,,,,,,,,,
2020-12-28 02:00:00,16.161650,2,0,30,0,363,53,12,4,2020,...,0,0,1,0,1,0,0,0,0,0
2020-12-28 02:30:00,12.645497,2,30,30,0,363,53,12,4,2020,...,0,0,1,0,1,0,0,0,0,0
2020-12-28 03:00:00,10.923170,3,0,30,0,363,53,12,4,2020,...,0,0,1,0,1,0,0,0,0,0
2020-12-28 03:30:00,10.517880,3,30,30,0,363,53,12,4,2020,...,0,0,1,0,1,0,0,0,0,0
2020-12-28 04:00:00,3.072320,4,0,30,0,363,53,12,4,2020,...,0,0,1,0,1,0,0,0,0,0


In [5]:
X = df.drop(['ama'], axis=1)
y = df['ama']

train_split_date = '2023-04-30'
test_split_date = '2023-09-11'

X_train = X.loc[X.index<=train_split_date]
X_test = X.loc[X.index>=test_split_date]

y_train = y.loc[y.index<=train_split_date]
y_test = y.loc[y.index>=test_split_date]

print("Size of X_train:", X_train.shape)
print("Size of y_train:", y_train.shape)

print("Size of X_test:", X_test.shape)
print("Size of y_test:", y_test.shape)

Size of X_train: (39703, 126)
Size of y_train: (39703,)
Size of X_test: (9695, 126)
Size of y_test: (9695,)


In [ ]:
!pip install xgboost

In [6]:
!pip install optuna

  Using cached optuna-3.6.1-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached Mako-1.3.5-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-3.6.1-py3-none-any.whl (380 kB)
Using cached alembic-1.13.1-py3-none-any.whl (233 kB)
Using cached colorlog-6.8.2-py3-none-any.whl (11 kB)
Using cached Mako-1.3.5-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [8]:
# import optuna
# import xgboost as xgb
# from sklearn.metrics import mean_absolute_error

# def objective(trial):
#     # Define the hyperparameters to tune
#     param = {
#         'verbosity': 0,
#         'objective': 'reg:absoluteerror',
#         'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
#         'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
#         'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
#         'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
#         'eta': trial.suggest_float('eta', 0.01, 0.2),
#         # 'tree_method': trial.suggest_categorical('tree_method', ['auto', 'exact', 'approx', 'hist', 'gpu_hist']),
#         'sampling_method': 'uniform',
#     }

#     if param['booster'] in ['gbtree', 'dart']:
#         param['max_depth'] = trial.suggest_int('max_depth', 1, 9)
#         param['gamma'] = trial.suggest_float('gamma', 1e-8, 1.0, log=True)
#         param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])

#     if param['booster'] == 'dart':
#         param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
#         param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
#         param['rate_drop'] = trial.suggest_float('rate_drop', 0.0, 1.0)
#         param['skip_drop'] = trial.suggest_float('skip_drop', 0.0, 1.0)

#     # Create and train the model
#     model = xgb.XGBRegressor(**param, early_stopping_rounds=50)
#     model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

#     # Predict and calculate MAE
#     preds = model.predict(X_test)
#     mae = mean_absolute_error(y_test, preds)
    
#     return mae


# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)  # Adjust n_trials as needed

# best_params = study.best_params
# print("Best parameters:", best_params)

# best_model = xgb.XGBRegressor(**best_params)
# best_model.fit(X_train, y_train)
# predictions = best_model.predict(X_test)
# final_mae = mean_absolute_error(y_test, predictions)
# print("Final MAE:", final_mae)

## Best performer

https://notebooks.cloud.google.com/view/tf-mnb-bucket-for_gang_huang/executor_files/final_xgboost_gh_install_xgboost/final_xgboost_gh_at_2024_05_26_09_08_18.ipynb?_ga=2.173138790.-472508634.1710958628

In [9]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

# # Best parameters as obtained from your tuning
# best_params = {
#     'booster': 'gbtree',
#     'lambda': 0.10789604170982146,
#     'alpha': 0.011973268750720061,
#     'subsample': 0.7114597091729832,
#     'colsample_bytree': 0.8449372802141699,
#     'min_child_weight': 2.455203448711688,
#     'n_estimators': 277,
#     'eta': 0.13532650973616234,
#     'max_depth': 4,
#     'gamma': 3.458865659897043e-05,
#     'grow_policy': 'lossguide'
# }

best_params = {
    'booster': 'gbtree',
    'lambda': 0.0001682885533435929,
    'alpha': 1.4545349952166233e-08,
    'subsample': 0.8001301757519088,
    'colsample_bytree': 0.8286111132463208,
    'min_child_weight': 3.19409978105852,
    'n_estimators': 196,
    'eta': 0.13124391559023546,
    'max_depth': 4,
    'gamma': 5.034688998428794e-08,
    'grow_policy': 'depthwise'
}


# Assuming X_train, y_train, X_test, and y_test are already defined in your workspace
# Create the XGBRegressor model with the best parameters
model = xgb.XGBRegressor(
    verbosity=0,  # Keeping the training logs quiet
    objective='reg:absoluteerror',  # As you focused on MAE
    **best_params
)

# Fit the model
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=50)

# Make predictions
predictions = model.predict(X_test)

# Calculate the Mean Absolute Error

# final_mae = mean_absolute_error(y_test, predictions)
# print("Final MAE on test set:", final_mae)

print("MAE on Test Set:", round(mean_absolute_error(y_test, predictions),5))
print("MSE on Test Set:", round(mean_squared_error(y_test, predictions),5))
print("RMSE on Test Set:", round(np.sqrt(mean_squared_error(y_test, predictions)),5))
print("R2 on Test Set:", round(r2_score(y_test, predictions),5))
print("MAPE on Test Set:", round(mean_absolute_percentage_error(y_test, predictions),5)) 

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


MAE on Test Set: 11.62972
MSE on Test Set: 232.4717
RMSE on Test Set: 15.24702
R2 on Test Set: 0.30553
MAPE on Test Set: 1845812048640992.0


In [10]:
# Predictions on training
preds_train = model.predict(X_train)

print("MAE on Train Set:", round(mean_absolute_error(y_train, preds_train),5))
print("MSE on Train Set:", round(mean_squared_error(y_train, preds_train),5))
print("RMSE on Train Set:", round(np.sqrt(mean_squared_error(y_train, preds_train)),5))
print("R2 on Train Set:", round(r2_score(y_train, preds_train),5))
print("MAPE on Train Set:", round(mean_absolute_percentage_error(y_train, preds_train),5)) 

MAE on Train Set: 12.09632
MSE on Train Set: 274.30444
RMSE on Train Set: 16.56214
R2 on Train Set: 0.45727
MAPE on Train Set: 554410988308164.44


In [16]:
X_test['ama'] = y_test
X_test['y_pred'] = predictions

/tmp/ipykernel_1/3211745372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['ama'] = y_test
/tmp/ipykernel_1/3211745372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['y_pred'] = predictions


In [17]:
X_test['abs_error'] = abs(X_test['ama']-X_test['y_pred'])

/tmp/ipykernel_1/2166114081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['abs_error'] = abs(X_test['ama']-X_test['y_pred'])


In [18]:
X_test[X_test.tod_prime==1]['abs_error'].mean()

14.834053451309595